
# Who said it : Using the Twitter API and logistic regression to see which tweets are more likely to be hillarys or trump 


## Goals
---

Given two people We are going to attempt to classify which tweet came from whom.  There are a few steps to this:
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset
- using supervised classification techniques


In [78]:
import numpy as np
import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        'L4sziHBqV4VUIfKezbos0JMVl',
    'consumer_secret':     'lJau6R7GIHFwoGR5wB3PlLQPXBChwzJFJ9WGXXtazcDSA1Vb1X',
    'access_token_key':    '941359629606539264-05XcmQfdwMXTbPNWS3r7cZThvbQBxCK',
    'access_token_secret': 'VdE3VJVk6oxbohQGcw7WYA5Tg4Sr8kW9duTO1wxmB6qXk'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode = 'extended'
)

In [79]:
type(api)

twitter.api.Api

### Below I use the GetUserTimeline method of twitter.api.Api object to get tweet text and tweet details for user. 

In [80]:
# Note: Count can't be greater than 200
x = api.GetUserTimeline(screen_name="HillaryClinton", count=20, include_rts=False,max_id=954430425321046016)
x = [_.AsDict() for _ in x]

In [81]:
x[0]

{u'created_at': u'Fri Jan 19 19:08:40 +0000 2018',
 u'favorite_count': 90621,
 u'full_text': u'I\u2019m so heartened by all of you. Onward! https://t.co/vkRj7M3kBS',
 u'hashtags': [],
 u'id': 954430425321046016,
 u'id_str': u'954430425321046016',
 u'lang': u'en',
 u'quoted_status': {u'created_at': u'Thu Jan 18 14:06:08 +0000 2018',
  u'favorite_count': 10541,
  u'full_text': u'New @TIME cover: A year ago, they marched. Now a record number of women are running for office https://t.co/Uc9ivXGS2q via @CharlotteAlter https://t.co/8jnQ9gWmur',
  u'hashtags': [],
  u'id': 953991903250386944,
  u'id_str': u'953991903250386944',
  u'lang': u'en',
  u'media': [{u'display_url': u'pic.twitter.com/8jnQ9gWmur',
    u'expanded_url': u'https://twitter.com/TIMEPolitics/status/953991903250386944/photo/1',
    u'id': 953991895411318784,
    u'media_url': u'http://pbs.twimg.com/media/DT1CkwyX4AAkH5W.jpg',
    u'media_url_https': u'https://pbs.twimg.com/media/DT1CkwyX4AAkH5W.jpg',
    u'sizes': {u'large':

In [82]:
#printing id and text of tweets
for element in x:
    print element['id']
    print(element['full_text'])
    print('--')

954430425321046016
I’m so heartened by all of you. Onward! https://t.co/vkRj7M3kBS
--
953005910930153472
These words from Dr. King also come to mind today: https://t.co/0qFK3RxBAF
--
952958227825782784
Beautifully said, @BerniceKing. An important message today and every day. https://t.co/eYJAGc6i2b
--
951895239140298752
The anniversary of the devastating earthquake 8 years ago is a day to remember the tragedy, honor the resilient people of Haiti, &amp; affirm America’s commitment to helping our neighbors. Instead, we‘re subjected to Trump’s ignorant, racist views of anyone who doesn’t look like him.
--
951651923328987136
@NancyEMcFadden Nancy has a record of beating the odds - from DC to CA - where she helped Gov Brown do a fantastic job. All who know her are sending strength &amp; love as she faces this latest challenge.  Onward my friend. -H
--
948244463138328577
Families across America had to start 2018 worried that their kids wouldn’t have health care. Failing to act now shows the 

In [83]:
# max id returns results with an ID less than (older than) or equal to the
#specified ID. 
y = api.GetUserTimeline(screen_name="HillaryClinton", count=20, max_id=933449899487780864, include_rts=False)
y = [_.AsDict() for _ in y]

In [84]:
for element in y:
    print element['id']
    print(element['full_text'])
    print('--')

933449899487780864
You go girl! This is important; costs will go up, &amp; powerful companies will get more powerful. We can’t let it slip through the cracks. https://t.co/js4agpzKeS
--
933107180131282944
And on top of that, you can’t beat a little book signing wardrobe fun.

https://t.co/K8VldCmTC6 https://t.co/9q9sTlpL3I
--
933105980484747264
And being out on the book tour it has been wonderful to hear from so many people about their activism, their courage, &amp; their resilience.
https://t.co/LhtTwV7N0d
--
933105682378870784
Wow. I wasn’t sure how letting my guard down would go...but it’s been cathartic &amp; rewarding. I loved writing this book, &amp; I’m honored to be in such great company on this list!
https://t.co/rAYpWHzxrJ
--
932746529475055618
It’s been almost 2 months since Congress let the program that provides health care for 9 million children expire. Call Congress &amp; tell them to reauthorize CHIP now: 202-224-3121 https://t.co/jk1FhOcMp2
--
931292276709580802
It’s tr

In [85]:
type(y[0])

dict

In [86]:
y[0]['id']

933449899487780864

In [1]:


class TweetMiner(object):

    
    def __init__(self, api, result_limit = 20):
        
        self.api = api        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="HillaryClinton", mine_retweets=False, max_pages=20):

        data           =  []
        last_tweet_id  =  954541219970977793
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id, include_rts=mine_retweets)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                last_tweet_id = item['id'] - 1
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---



In [88]:
# Result limit == count parameter from our GetUserTimeline()
miner = TweetMiner(api, result_limit=200)

In [89]:
hillary = miner.mine_user_tweets(user="HillaryClinton")
donald = miner.mine_user_tweets(user="realDonaldTrump")

In [90]:
for x in range(5):
    print hillary[x]['text']
    print('---')

I’m so heartened by all of you. Onward! https://t.co/vkRj7M3kBS
---
These words from Dr. King also come to mind today: https://t.co/0qFK3RxBAF
---
Beautifully said, @BerniceKing. An important message today and every day. https://t.co/eYJAGc6i2b
---
The anniversary of the devastating earthquake 8 years ago is a day to remember the tragedy, honor the resilient people of Haiti, &amp; affirm America’s commitment to helping our neighbors. Instead, we‘re subjected to Trump’s ignorant, racist views of anyone who doesn’t look like him.
---
@NancyEMcFadden Nancy has a record of beating the odds - from DC to CA - where she helped Gov Brown do a fantastic job. All who know her are sending strength &amp; love as she faces this latest challenge.  Onward my friend. -H
---


In [91]:
for x in range(5):
    print donald[x]['text']
    print('--')

Not looking good for our great Military or Safety &amp; Security on the very dangerous Southern Border. Dems want a Shutdown in order to help diminish the great success of the Tax Cuts, and what they are doing for our booming economy.
--
Excellent preliminary meeting in Oval with @SenSchumer - working on solutions for Security and our great Military together with @SenateMajLdr McConnell and @SpeakerRyan. Making progress - four week extension would be best!
--
Just signed 702 Bill to reauthorize foreign intelligence collection. This is NOT the same FISA law that was so wrongly abused during the election. I will always do the right thing for our country and put the safety of the American people first!
--
Today, I was honored and proud to address the 45th Annual @March_for_Life! You are living witnesses of this year’s March for Life theme: #LoveSavesLives. https://t.co/DMST4qhDmp
--
“Shutting down the government is a very serious thing. People die, accidents happen. I don’t know how I wou

### Convert the tweet ouputs to a pandas DataFrame

In [92]:
# A:
pd.DataFrame(hillary)

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Fri Jan 19 19:08:40 +0000 2018,HillaryClinton,2020-05-31 02:08:40.529365,16059,I’m so heartened by all of you. Onward! https:...,954430425321046016
1,Mon Jan 15 20:48:09 +0000 2018,HillaryClinton,2020-05-31 02:08:40.529391,23021,These words from Dr. King also come to mind to...,953005910930153472
2,Mon Jan 15 17:38:41 +0000 2018,HillaryClinton,2020-05-31 02:08:40.529402,8990,"Beautifully said, @BerniceKing. An important m...",952958227825782784
3,Fri Jan 12 19:14:45 +0000 2018,HillaryClinton,2020-05-31 02:08:40.529413,59635,The anniversary of the devastating earthquake ...,951895239140298752
4,Fri Jan 12 03:07:54 +0000 2018,HillaryClinton,2020-05-31 02:08:40.529421,320,@NancyEMcFadden Nancy has a record of beating ...,951651923328987136
5,Tue Jan 02 17:27:52 +0000 2018,HillaryClinton,2020-05-31 02:08:40.529430,8529,Families across America had to start 2018 worr...,948244463138328577
6,Tue Jan 02 17:26:39 +0000 2018,HillaryClinton,2020-05-31 02:08:40.529438,14714,Time to bring CHIP to the Senate floor as prom...,948244159986651136
7,Sun Dec 31 03:49:33 +0000 2017,HillaryClinton,2020-05-31 02:08:40.529447,20666,"The Iranian people, especially the young, are ...",947313751992274944
8,Fri Dec 22 18:53:08 +0000 2017,HillaryClinton,2020-05-31 02:08:40.529454,3153,Thank you to everyone who has donated to Onwar...,944279653883228160
9,Fri Dec 22 18:52:12 +0000 2017,HillaryClinton,2020-05-31 02:08:40.529462,2478,"Along with @IndivisibleTeam, @ColorOfChange, @...",944279421065814016


##  Create the training data

---

"Mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Hillary tweets
- Append the results to our DataFrame

In [93]:
# A:
miner = TweetMiner(api, result_limit=200)
trump_tweets = miner.mine_user_tweets("realDonaldTrump", max_pages=14)

In [94]:
trump_df = pd.DataFrame(trump_tweets)

In [95]:
trump_df

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Sat Jan 20 02:28:56 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242658,20046,Not looking good for our great Military or Saf...,954541219970977793
1,Fri Jan 19 22:17:53 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242677,10552,Excellent preliminary meeting in Oval with @Se...,954478044487520257
2,Fri Jan 19 20:53:17 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242683,21828,Just signed 702 Bill to reauthorize foreign in...,954456754137501697
3,Fri Jan 19 18:39:50 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242687,18452,"Today, I was honored and proud to address the ...",954423170299199488
4,Fri Jan 19 16:28:29 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242692,13149,“Shutting down the government is a very seriou...,954390114968498176
5,Fri Jan 19 16:14:05 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242698,7464,.@WhiteHouse Briefing with Director Marc Short...,954386490842378241
6,Fri Jan 19 12:04:47 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242702,24203,Government Funding Bill past last night in the...,954323750949982208
7,Thu Jan 18 23:39:53 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242707,19717,House of Representatives needs to pass Governm...,954136290768846850
8,Thu Jan 18 21:04:36 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242712,24458,AMERICA will once again be a NATION that think...,954097213608570880
9,Thu Jan 18 20:45:33 +0000 2018,realDonaldTrump,2020-05-31 02:09:09.242717,14567,"Departing Pittsburgh now, where it was my grea...",954092417250222082


In [96]:
trump_df.shape

(2479, 6)

In [97]:
hillary_tweets = miner.mine_user_tweets('HillaryClinton')

In [98]:
hillary_df = pd.DataFrame(hillary_tweets)
print hillary_df.shape

(2567, 6)


In [99]:
tweets = pd.concat([trump_df, hillary_df], axis=0)
tweets.shape

(5046, 6)

## Any interesting ngrams going on with Trump or Hillary?

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'fake news', 162),
 (u'america great', 79),
 (u'tax cuts', 72),
 (u'make america', 65),
 (u'united states', 63),
 (u'make america great', 59),
 (u'north korea', 50),
 (u'news media', 44),
 (u'white house', 43),
 (u'great honor', 43),
 (u'fake news media', 42),
 (u'stock market', 40),
 (u'tax cut', 34),
 (u'working hard', 32),
 (u'usa https', 31),
 (u'hillary clinton', 30),
 (u'puerto rico', 27),
 (u'prime minister', 27),
 (u'american people', 27),
 (u'jobs jobs', 26)]

## Fake news....figures

In [101]:
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

summaries = "".join(hillary_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[(u'donald trump', 218),
 (u'hillary https', 101),
 (u'https ttgeqxnqym', 88),
 (u'hillary clinton', 69),
 (u'vote https', 57),
 (u'make sure', 52),
 (u'trump https', 48),
 (u'ttgeqxnqym https', 47),
 (u'https ttgeqxnqym https', 47),
 (u'debatenight https', 47),
 (u'potus https', 45),
 (u'https 3tkj4h68kz', 44),
 (u'president https', 39),
 (u'https 3tkj4h68kz https', 32),
 (u'3tkj4h68kz https', 32),
 (u'united states', 31),
 (u'flotus https', 30),
 (u've got', 30),
 (u'commander chief', 29),
 (u'donald trump https', 28)]

## Processing the tweets and building a model

---

To do classfication I will need to convert the tweets into a set of features.

**I'll do this by:**
- Vectorizing input text data.
- Intializing a model.
- Grid Searching for optimal hyperparameters.
- Training and fitting optimized model.
- Evaluating the performance of the model.

In [102]:
#cleaning text data using textacy
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [103]:
print tweet_text[1:8]

[ u'Excellent preliminary meeting in Oval with @SenSchumer - working on solutions for Security and our great Military together with @SenateMajLdr McConnell and @SpeakerRyan. Making progress - four week extension would be best!'
 u'Just signed 702 Bill to reauthorize foreign intelligence collection. This is NOT the same FISA law that was so wrongly abused during the election. I will always do the right thing for our country and put the safety of the American people first!'
 u'Today, I was honored and proud to address the 45th Annual @March_for_Life! You are living witnesses of this year\u2019s March for Life theme: #LoveSavesLives. https://t.co/DMST4qhDmp'
 u'\u201cShutting down the government is a very serious thing. People die, accidents happen. I don\u2019t know how I would vote right now on a CR, OK?\u201d\nSen. Dianne Feinstein (D-Calif)\nhttps://t.co/7xP3CBnv5j'
 u'.@WhiteHouse Briefing with Director Marc Short and Director Mick Mulvaney...\nhttps://t.co/0O0VsYXmHB'
 u'Government 

In [104]:
print clean_text[1:8]

[u'excellent preliminary meeting in oval with senschumer working on solutions for security and our great military together with senatemajldr mcconnell and speakerryan making progress four week extension would be best', u'just signed 702 bill to reauthorize foreign intelligence collection this is not the same fisa law that was so wrongly abused during the election i will always do the right thing for our country and put the safety of the american people first', u'today i was honored and proud to address the 45th annual marchforlife you are living witnesses of this years march for life theme lovesaveslives url', u'shutting down the government is a very serious thing people die accidents happen i dont know how i would vote right now on a cr ok\nsen dianne feinstein dcalif\nurl', u'whitehouse briefing with director marc short and director mick mulvaney\nurl', u'government funding bill past last night in the house of representatives now democrats are needed if it is to pass in the senate bu

In [105]:
#creating target
y = tweets['handle'].map(lambda x: 1 if x == 'realDonaldTrump' else 0).values
print max(pd.Series(y).value_counts(normalize=True))

0.508719778042


In [106]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


#Vectorizing with TF-IDF Vectorizer and creating X matrix
tfv = TfidfVectorizer(ngram_range=(2,4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()
print X.shape

(5046, 2000)


In [107]:
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression()
params = {'penalty': ['l1', 'l2'], 'C':np.logspace(-5,0,100)}
#Grid searching to find optimal parameters for Logistic Regression
gs = GridSearchCV(lr, param_grid=params, cv=10, verbose=1)
gs.fit(X, y)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed:  2.2min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([  1.00000e-05,   1.12332e-05, ...,   8.90215e-01,   1.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [108]:
print gs.best_params_
print gs.best_score_

{'penalty': 'l2', 'C': 1.0}
0.85037653587


## Check the predicted probability for a random hillary and Trump tweet
---

Below are provided a couple of tweets from both Hillary and Trump.

In [45]:
estimator = LogisticRegression(penalty='l2',C=1.0)
estimator.fit(X,y)

# Prep our source as TfIdf vectors
source_test = [
    "The presidency doesn’t change who you are—it reveals who you are. And we’ve seen all we need to of Donald Trump.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]


Xtest = tfv.transform(source_test)
pd.DataFrame(estimator.predict_proba(Xtest), columns=["Proba_Hillary", "Proba_Trump"])

,Proba_Hillary,Proba_Trump
0,0.899186,0.100814
1,0.313876,0.686124


So based on the model, the probablility the first tweet came from Hillary is almost 90%! The probability the second tweet came from Trump is almost 70%. So I would say the model is performing pretty well.